## DataScience Challenge 4
# _ StarCraft _
![StarCraft](sc2_guide_generaliste.jpg "StarCraft 2")

Starcraft est un jeu video de strategie en temps réel pratiqué lors de compétitions internationnales de sport électronique. 

Il permet à deux joueurs de s'affronter pour la domination militaire du territoire. Au début du jeu, chaque joueur choisit une des trois races disponibles qui ont différentes capacitées et styles de jeu. Les joueurs doivent récolter des ressources pour produire des batiments et des unitées de combat. 

![gameplay](611he05az8L.jpg "Gameplay")

** L'objectif de ce challenge est de prédire le vainqueur d'une partie de StarCraft **


## 1 - Les données

* Les données des matches ont été téléchargées sur [ggtracker](http://ggtracker.com/matches), une plateforme d’analyse statistique des replays des joueurs inscrits. 

* Les actions des joueurs ont été extraites à l’aide de l’utilitaire [sc2gears](https://github.com/icza/sc2gears). 

* Les fichiers texte ont été convertis au format csv à l’aide d’un script PowerShell. 

* Les données ont été extraites par un script Python. 

* Le jeu de données sc2_dataset_lt_5mins.csv est composé de **328** variables issues de **34534** matches joués. 

**La cible à prédire est le vainqueur du match indiqué dans la variable `_p1_win_game`.**

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import numpy as np
pd.set_option("display.max_rows",250)
pd.set_option("display.max_columns",250)
#Classifiers
from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF

In [3]:
dataset = pd.read_csv("sc2_dataset_lt_5mins.csv")

#Remplacement des valeurs manquantes
non_sparse_dataset = dataset.fillna(value=0.0)

#Quelle est la taille de la DataFrame ? 
print("Le jeu de données contient %d lignes et %d colonnes" % (non_sparse_dataset.shape[0], non_sparse_dataset.shape[1]))

Le jeu de données contient 34534 lignes et 328 colonnes


In [4]:
non_sparse_dataset.head()

,_p1_win_game,game_length,p1_APM,p1_APM_wo_select,p1_Archon,p1_Armory,p1_Assimilator,p1_Auto_Turret,p1_Baneling_Nest,p1_Banshee,p1_Barracks,p1_Battlecruiser,p1_Building_Attack,p1_Bunker,p1_Carrier,p1_Colossus,p1_Command_Center,p1_Corruptor,p1_Creep_Tumor,p1_Cybernetics_Core,p1_Dark,p1_Dark_Shrine,p1_Drone,p1_Engineering_Bay,p1_Evolution_Chamber,p1_Extractor,p1_Factory,p1_Fleet_Beacon,p1_Forge,p1_Fusion_Core,p1_Gateway,p1_Ghost,p1_Ghost_Academy,p1_Hatchery,p1_Hellion,p1_High,p1_Hive,p1_Hydralisk,p1_Hydralisk_Den,p1_Immortal,p1_Infestation_Pit,p1_Infestor,p1_Lair,p1_Lair_Upgrade_Hatchery,p1_Marauder,p1_Marine,p1_Medivac,p1_Missile_Turret,p1_Mothership,p1_Mutalisk,p1_Nexus,p1_Nydus_Network,p1_Nydus_Worm,p1_Observer,p1_Orbital_Command_Upgrade_Command_Center,p1_Overlord,p1_Phoenix,p1_Photon_Cannon,p1_Planetary_Fortress_Upgrade_Command_Center,p1_Point_Defense,p1_Probe,p1_Protoss_Air_Armor_1,p1_Protoss_Air_Armor_2,p1_Protoss_Air_Armor_3,p1_Protoss_Air_Weapons_1,p1_Protoss_Air_Weapons_2,p1_Protoss_Air_Weapons_3,p1_Protoss_Ground_Armor_1,p1_Protoss_Ground_Armor_2,p1_Protoss_Ground_Armor_3,p1_Protoss_Ground_Weapons_1,p1_Protoss_Ground_Weapons_2,p1_Protoss_Ground_Weapons_3,p1_Protoss_Shield_1,p1_Protoss_Shield_2,p1_Protoss_Shield_3,p1_Pylon,p1_Queen,p1_Raven,p1_Reactor_Barracks,p1_Reactor_Factory,p1_Reactor_Starport,p1_Reaper,p1_Refinery,p1_Roach,p1_Roach_Warren,p1_Robotics_Bay,p1_Robotics_Facility,p1_SCV,p1_Sensor_Tower,p1_Sentry,p1_Siege,p1_Spawning_Pool,p1_Spine_Crawler,p1_Spire,p1_Spore_Crawler,p1_Stalker,p1_Stargate,p1_Starport,p1_Supply_Depot,p1_Tech_Lab,p1_Templar_Archives,p1_Terran_Building,p1_Terran_Building_Armor,p1_Terran_Hi-sec_Auto_Tracking,p1_Terran_Infantry_Armor_1,p1_Terran_Infantry_Armor_2,p1_Terran_Infantry_Armor_3,p1_Terran_Infantry_Weapons_1,p1_Terran_Infantry_Weapons_2,p1_Terran_Infantry_Weapons_3,p1_Terran_Neosteel_Frame,p1_Terran_Ship_Plating_1,p1_Terran_Ship_Plating_2,p1_Terran_Ship_Plating_3,p1_Terran_Ship_Weapons_1,p1_Terran_Ship_Weapons_2,p1_Terran_Ship_Weapons_3,p1_Terran_Vehicle_Plating_1,p1_Terran_Vehicle_Plating_2,p1_Terran_Vehicle_Plating_3,p1_Terran_Vehicle_Weapons_1,p1_Terran_Vehicle_Weapons_2,p1_Terran_Vehicle_Weapons_3,p1_Thor,...,p2_Infestation_Pit,p2_Infestor,p2_Lair,p2_Lair_Upgrade_Hatchery,p2_Marauder,p2_Marine,p2_Medivac,p2_Missile_Turret,p2_Mothership,p2_Mutalisk,p2_Nexus,p2_Nydus_Network,p2_Nydus_Worm,p2_Observer,p2_Orbital_Command_Upgrade_Command_Center,p2_Overlord,p2_Phoenix,p2_Photon_Cannon,p2_Planetary_Fortress_Upgrade_Command_Center,p2_Point_Defense,p2_Probe,p2_Protoss_Air_Armor_1,p2_Protoss_Air_Armor_2,p2_Protoss_Air_Armor_3,p2_Protoss_Air_Weapons_1,p2_Protoss_Air_Weapons_2,p2_Protoss_Air_Weapons_3,p2_Protoss_Ground_Armor_1,p2_Protoss_Ground_Armor_2,p2_Protoss_Ground_Armor_3,p2_Protoss_Ground_Weapons_1,p2_Protoss_Ground_Weapons_2,p2_Protoss_Ground_Weapons_3,p2_Protoss_Shield_1,p2_Protoss_Shield_2,p2_Protoss_Shield_3,p2_Pylon,p2_Queen,p2_Raven,p2_Reactor_Barracks,p2_Reactor_Factory,p2_Reactor_Starport,p2_Reaper,p2_Refinery,p2_Roach,p2_Roach_Warren,p2_Robotics_Bay,p2_Robotics_Facility,p2_SCV,p2_Sensor_Tower,p2_Sentry,p2_Siege,p2_Spawning_Pool,p2_Spine_Crawler,p2_Spire,p2_Spore_Crawler,p2_Stalker,p2_Stargate,p2_Starport,p2_Supply_Depot,p2_Tech_Lab,p2_Templar_Archives,p2_Terran_Building,p2_Terran_Building_Armor,p2_Terran_Hi-sec_Auto_Tracking,p2_Terran_Infantry_Armor_1,p2_Terran_Infantry_Armor_2,p2_Terran_Infantry_Armor_3,p2_Terran_Infantry_Weapons_1,p2_Terran_Infantry_Weapons_2,p2_Terran_Infantry_Weapons_3,p2_Terran_Neosteel_Frame,p2_Terran_Ship_Plating_1,p2_Terran_Ship_Plating_2,p2_Terran_Ship_Plating_3,p2_Terran_Ship_Weapons_1,p2_Terran_Ship_Weapons_2,p2_Terran_Ship_Weapons_3,p2_Terran_Vehicle_Plating_1,p2_Terran_Vehicle_Plating_2,p2_Terran_Vehicle_Plating_3,p2_Terran_Vehicle_Weapons_1,p2_Terran_Vehicle_Weapons_2,p2_Terran_Vehicle_Weapons_3,p2_Thor,p2_Twilight_Council,p2_Ultralisk,p2_Ultralisk_Cavern,p2_Viking,p2_Void,p2_Warp,p2_Zealot,p2_Zerg_Flyer_Attacks_1,p2_Zerg_Flyer_Attacks_2,p2_Zerg_

`APM` : Actions par minutes. Plus cette variable est élevée et plus le joueur est expérimenté. 

In [9]:
%%HTML
<video width="640" height="480" controls>
    <source src="Jaedong_APM.mp4" type="video/mp4">
</video>

## 2 - Feature engineering

Nous voulons comparer les actions des deux joueurs pour prédire la victoire ou la défaite du joueur 1. Nous allons donc créer des variables qui quantifient les différences entre les deux joueurs. Les variables de chaque joueurs sont identifiées avec les indicateurs `p1_` et  `p2_`.


In [5]:
#la cible et la durée du match sont exclues du dataset
df = non_sparse_dataset.iloc[:,2:]

#Les noms des variables sans leurs indicateurs de joueurs sont collectés
#collecte de noms
colnames = df.columns.values
#noms uniques suffixé par '$'
colnames = list(set([n[2:]+'$' for n in colnames]))
colnames.sort()

#Création des variables correspondants à la différence entre p1_ et p2_
select_df = pd.DataFrame()

for name in colnames:
    filt_df = df.filter(regex=name)
    
    #nom de colonne sans l'expression regulière de fin de mot '$'
    select_df[name[:-1]] = filt_df.iloc[:,0] - filt_df.iloc[:,1]


In [6]:
select_df.head()

,_APM,_APM_wo_select,_Archon,_Armory,_Assimilator,_Auto_Turret,_Baneling_Nest,_Banshee,_Barracks,_Battlecruiser,_Building_Attack,_Bunker,_Carrier,_Colossus,_Command_Center,_Corruptor,_Creep_Tumor,_Cybernetics_Core,_Dark,_Dark_Shrine,_Drone,_Engineering_Bay,_Evolution_Chamber,_Extractor,_Factory,_Fleet_Beacon,_Forge,_Fusion_Core,_Gateway,_Ghost,_Ghost_Academy,_Hatchery,_Hellion,_High,_Hive,_Hydralisk,_Hydralisk_Den,_Immortal,_Infestation_Pit,_Infestor,_Lair,_Lair_Upgrade_Hatchery,_Marauder,_Marine,_Medivac,_Missile_Turret,_Mothership,_Mutalisk,_Nexus,_Nydus_Network,_Nydus_Worm,_Observer,_Orbital_Command_Upgrade_Command_Center,_Overlord,_Phoenix,_Photon_Cannon,_Planetary_Fortress_Upgrade_Command_Center,_Point_Defense,_Probe,_Protoss_Air_Armor_1,_Protoss_Air_Armor_2,_Protoss_Air_Armor_3,_Protoss_Air_Weapons_1,_Protoss_Air_Weapons_2,_Protoss_Air_Weapons_3,_Protoss_Ground_Armor_1,_Protoss_Ground_Armor_2,_Protoss_Ground_Armor_3,_Protoss_Ground_Weapons_1,_Protoss_Ground_Weapons_2,_Protoss_Ground_Weapons_3,_Protoss_Shield_1,_Protoss_Shield_2,_Protoss_Shield_3,_Pylon,_Queen,_Raven,_Reactor_Barracks,_Reactor_Factory,_Reactor_Starport,_Reaper,_Refinery,_Roach,_Roach_Warren,_Robotics_Bay,_Robotics_Facility,_SCV,_Sensor_Tower,_Sentry,_Siege,_Spawning_Pool,_Spine_Crawler,_Spire,_Spore_Crawler,_Stalker,_Stargate,_Starport,_Supply_Depot,_Tech_Lab,_Templar_Archives,_Terran_Building,_Terran_Building_Armor,_Terran_Hi-sec_Auto_Tracking,_Terran_Infantry_Armor_1,_Terran_Infantry_Armor_2,_Terran_Infantry_Armor_3,_Terran_Infantry_Weapons_1,_Terran_Infantry_Weapons_2,_Terran_Infantry_Weapons_3,_Terran_Neosteel_Frame,_Terran_Ship_Plating_1,_Terran_Ship_Plating_2,_Terran_Ship_Plating_3,_Terran_Ship_Weapons_1,_Terran_Ship_Weapons_2,_Terran_Ship_Weapons_3,_Terran_Vehicle_Plating_1,_Terran_Vehicle_Plating_2,_Terran_Vehicle_Plating_3,_Terran_Vehicle_Weapons_1,_Terran_Vehicle_Weapons_2,_Terran_Vehicle_Weapons_3,_Thor,_Twilight_Council,_Ultralisk,_Ultralisk_Cavern,_Viking,_Void,_Warp,_Zealot,_Zerg_Flyer_Attacks_1,_Zerg_Flyer_Attacks_2,_Zerg_Flyer_Attacks_3,_Zerg_Flyer_Carapace_1,_Zerg_Flyer_Carapace_2,_Zerg_Flyer_Carapace_3,_Zerg_Ground_Carapace_1,_Zerg_Ground_Carapace_2,_Zerg_Ground_Carapace_3,_Zerg_Melee_Attacks_1,_Zerg_Melee_Attacks_2,_Zerg_Melee_Attacks_3,_Zerg_Missile_Attacks_1,_Zerg_Missile_Attacks_2,_Zerg_Missile_Attacks_3,_Zergling,_an,_assigned_hotkeys,_building_types,_collected_gas,_collected_minerals,_max_distance_from_opponent,_mean_distance_from_opponent,_min_distance_from_opponent,_nb_scooting,_selected_hotkeys,_spent_gas,_spent_minerals,_timing_1st_scooting,_to_Orbital_Command_Command_Center,_to_Planetary_Fortress_Command_Center,_to_Warp_Gate_Gateway,_unit_types
0,-52.514968,-24.194918,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-2.0,0.0,-21,1,-98.407317,3178.767457,8.921409,13.573083,2.755987,-25,-396,1025,2725,-41.638376,0.0,0.0,0.0,0
1,47.431682,13.239911,0.0,-2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0,40.0,0.0,0.0,0.0,127.0,-2.0,2.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,5.0,-6.0,0.0,1.0,0.0,0.0,0.0,1.0,9.0,2.0,0.0,-12.0,-119.0,-4.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,-4.0,0.0,-2.0,0.0,-6.0,9.0,1.0,0.0,0.0,-72.0,0.0,0.0,-19.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,-20.0,-9.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,